## Prediksi Penyakit Dengan Machine Learning

# Import Library dan Dataset

Pertama, saya memuat pustaka-pustaka yang diperlukan dan dataset dari file CSV menggunakan pandas. Dataset ini terdiri dari data latih dan data uji untuk melatih dan menguji model prediksi penyakit.

In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from statistics import mode
%matplotlib inline

# Pemeriksaan Keseimbangan Dataset

Setelah memuat dataset, langkah selanjutnya adalah memeriksa apakah dataset seimbang atau tidak. Ini dilakukan dengan memplot jumlah sampel untuk setiap penyakit yang terdaftar dalam kolom "prognosis".

In [ ]:
# Membaca dataset dari file CSV dan menghapus kolom kosong
DATA_PATH = "Training.csv"
data = pd.read_csv(DATA_PATH).dropna(axis = 1)

# Memeriksa keseimbangan dataset dengan plot bar
disease_counts = data["prognosis"].value_counts()
temp_df = pd.DataFrame({
    "Disease": disease_counts.index,
    "Counts": disease_counts.values
})

plt.figure(figsize = (18,8))
sns.barplot(x = "Disease", y = "Counts", data = temp_df)
plt.xticks(rotation=90)
plt.show()

# Preprocessing Data

Setelah memastikan dataset seimbang, langkah berikutnya adalah melakukan preprocessing terhadap data. Hal ini meliputi mengubah kolom target ("prognosis") dari tipe string ke tipe numerik menggunakan Label Encoder, serta membagi data menjadi data latih dan data uji untuk evaluasi model.

In [ ]:
# Mengubah nilai target menjadi nilai numerik menggunakan LabelEncoder
encoder = LabelEncoder()
data["prognosis"] = encoder.fit_transform(data["prognosis"])

# Membagi dataset menjadi data latih dan data uji

In [ ]:
X = data.iloc[:,:-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test =train_test_split(
  X, y, test_size = 0.2, random_state = 24)

print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")

# Pembangunan Model

Setelah mempersiapkan data, langkah selanjutnya adalah membangun model menggunakan tiga algoritma yang berbeda: Support Vector Classifier (SVC), Gaussian Naive Bayes Classifier, dan Random Forest Classifier. Saya menggunakan K-Fold Cross-Validation untuk mengevaluasi kinerja masing-masing model.

In [ ]:
# Mendefinisikan metrik penilaian untuk cross-validation k-fold
def cv_scoring(estimator, X, y):
    return accuracy_score(y, estimator.predict(X))

# Inisialisasi Model
models = {
    "SVC":SVC(),
    "Gaussian NB":GaussianNB(),
    "Random Forest":RandomForestClassifier(random_state=18)
}

# Evaluasi model menggunakan K-Fold Cross-Validation
for model_name in models:
    model = models[model_name]
    scores = cross_val_score(model, X, y, cv = 10,
                             n_jobs = -1,
                             scoring = cv_scoring)
    print("=="*30)
    print(model_name)
    print(f"Scores: {scores}")
    print(f"Mean Score: {np.mean(scores)}")

#Kombinasi Model dan Evaluasi

Untuk membangun model yang lebih robust, saya mengkombinasikan prediksi dari ketiga model yang telah dilatih. Hal ini dilakukan dengan melatih setiap model pada data latih dan mengevaluasi kinerjanya menggunakan matriks pada data uji.

In [ ]:
# Melatih dan menguji SVM Classifier
svm_model = SVC()
svm_model.fit(X_train, y_train)
preds = svm_model.predict(X_test)

print(f"Accuracy on train data by SVM Classifier: {accuracy_score(y_train, svm_model.predict(X_train)) * 100}")
print(f"Accuracy on test data by SVM Classifier: {accuracy_score(y_test, preds) * 100}")

cf_matrix = confusion_matrix(y_test, preds)
plt.figure(figsize=(12, 8))
sns.heatmap(cf_matrix, annot=True)
plt.title("Confusion Matrix for SVM Classifier on Test Data")
plt.show()

# Melatih dan menguji Naive Bayes Classifier
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
preds = nb_model.predict(X_test)

print(f"Accuracy on train data by Naive Bayes Classifier: {accuracy_score(y_train, nb_model.predict(X_train)) * 100}")
print(f"Accuracy on test data by Naive Bayes Classifier: {accuracy_score(y_test, preds) * 100}")

cf_matrix = confusion_matrix(y_test, preds)
plt.figure(figsize=(12, 8))
sns.heatmap(cf_matrix, annot=True)
plt.title("Confusion Matrix for Naive Bayes Classifier on Test Data")
plt.show()

# Melatih dan menguji Random Forest Classifier
rf_model = RandomForestClassifier(random_state=18)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

print(f"Accuracy on train data by Random Forest Classifier: {accuracy_score(y_train, rf_model.predict(X_train)) * 100}")
print(f"Accuracy on test data by Random Forest Classifier: {accuracy_score(y_test, preds) * 100}")

cf_matrix = confusion_matrix(y_test, preds)
plt.figure(figsize=(12, 8))
sns.heatmap(cf_matrix, annot=True)
plt.title("Confusion Matrix for Random Forest Classifier on Test Data")
plt.show()

# Pengujian Pada Data Uji

Setelah memastikan kinerja model yang baik pada data uji, saya menguji model kombinasi pada dataset pengujian. Hasil prediksi diperoleh dengan menggabungkan prediksi dari SVM, Naive Bayes, dan Random Forest menggunakan mode dari prediksi-prediksi tersebut.

In [ ]:
# Melatih model pada seluruh data dan memvalidasi pada dataset pengujian
final_svm_model = SVC()
final_nb_model = GaussianNB()
final_rf_model = RandomForestClassifier(random_state=18)
final_svm_model.fit(X, y)
final_nb_model.fit(X, y)
final_rf_model.fit(X, y)

# Membaca data pengujian
test_data = pd.read_csv("Testing.csv").dropna(axis=1)

test_X = test_data.iloc[:, :-1]
test_Y = encoder.transform(test_data.iloc[:, -1])

# Melakukan prediksi dengan mengambil mode dari prediksi semua klasifier
svm_preds = final_svm_model.predict(test_X)
nb_preds = final_nb_model.predict(test_X)
rf_preds = final_rf_model.predict(test_X)

final_preds = mode([svm_preds, nb_preds, rf_preds], axis=0)[0].flatten()

print(f"Accuracy on test data by SVM Classifier: {accuracy_score(test_Y, svm_preds) * 100}")

cf_matrix = confusion_matrix(test_Y, final_preds)
plt.figure(figsize=(12, 8))
sns.heatmap(cf_matrix, annot=True)
plt.title("Confusion Matrix for Final Model on Test Data")
plt.show()

# Fungsi Prediksi
Terakhir, saya membuat fungsi yang dapat menerima gejala-gejala yang dipisahkan oleh koma sebagai input, dan menghasilkan prediksi penyakit menggunakan model gabungan berdasarkan gejala tersebut.



In [ ]:
symptoms = X.columns.values

symptom_index = {}
for index, value in enumerate(symptoms):
    symptom = " ".join([i.capitalize() for i in value.split("_")])
    symptom_index[symptom] = index

data_dict = {
    "symptom_index": symptom_index,
    "predictions_classes": encoder.classes_
}

# Membuat fungsi untuk menghasilkan prediksi penyakit berdasarkan gejala input
def predictDisease(symptoms):
    symptoms = symptoms.split(",")

    # Membuat data input untuk model
    input_data = [0] * len(data_dict["symptom_index"])
    for symptom in symptoms:
        symptom = symptom.strip()  # Menghapus spasi yang tidak diperlukan
        if symptom in data_dict["symptom_index"]:
            index = data_dict["symptom_index"][symptom]
            input_data[index] = 1

    # Mengubah data input menjadi DataFrame
    input_data = pd.DataFrame([input_data], columns=X.columns)

    # Melakukan prediksi dengan masing-masing model
    rf_prediction = data_dict["predictions_classes"][final_rf_model.predict(input_data)[0]]
    nb_prediction = data_dict["predictions_classes"][final_nb_model.predict(input_data)[0]]
    svm_prediction = data_dict["predictions_classes"][final_svm_model.predict(input_data)[0]]

    # Menggabungkan prediksi dengan mengambil mode dari semua prediksi
    final_prediction = mode([rf_prediction, nb_prediction, svm_prediction])
    predictions = {
        "rf_model_prediction": rf_prediction,
        "naive_bayes_prediction": nb_prediction,
        "svm_model_prediction": svm_prediction,
        "final_prediction": final_prediction
    }
    return predictions

# Menguji fungsi prediksi
print(predictDisease("Itching,Skin Rash,Nodal Skin Eruptions"))